In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np

cleanedCalendar = pd.read_csv("../cleanedData/cleanedCalendar.csv")
cleanedListings = pd.read_csv("../cleanedData/cleanedListings.csv")
cleanedReviews = pd.read_csv("../cleanedData/cleanedReviews.csv")

for df in [cleanedCalendar, cleanedListings, cleanedReviews]:
    df.columns = df.columns.str.strip().str.lower().str.replace(r"\s+", "_", regex=True)

# Listings is your base since each listing_id appears in the other two
finalizedData = cleanedListings.copy()

# Merge Calendar info
if "listing_id" in cleanedCalendar.columns:
    cleanedCalendar["listing_id"] = pd.to_numeric(cleanedCalendar["listing_id"], errors="coerce")
    finalizedData = finalizedData.merge(
        cleanedCalendar.groupby("listing_id").agg(
            cal_availability_rate=("available", "mean") if "available" in cleanedCalendar.columns else ("listing_id",
                                                                                                        "size")
        ).reset_index(),
        left_on="id", right_on="listing_id", how="left"
    ).drop(columns=["listing_id"], errors="ignore")

# Merge Reviews info
if "listing_id" in cleanedReviews.columns:
    cleanedReviews["listing_id"] = pd.to_numeric(cleanedReviews["listing_id"], errors="coerce")
    finalizedData = finalizedData.merge(
        cleanedReviews.groupby("listing_id").agg(
            review_count=("id", "count") if "id" in cleanedReviews.columns else ("listing_id", "size")
        ).reset_index(),
        left_on="id", right_on="listing_id", how="left"
    ).drop(columns=["listing_id"], errors="ignore")

drop_cols = [
    "id", "price", "cal_mean_price", "cal_median_price",
    "cal_min_price", "cal_max_price", "cal_priced_days",
    "first_review_date", "last_review_date", "avg_comment_len", "amenities",
    "availability_30", "availability_60", "availability_90", "availability_365"
]
finalizedData = finalizedData.drop(columns=[c for c in drop_cols if c in finalizedData.columns], errors="ignore")

finalizedData.to_csv("../finalizedData/finalizedData.csv", index=False)
print("✅ Final dataset saved as finalizedData/finalizedData.csv")

print(finalizedData.shape)
print(finalizedData.columns.tolist())

✅ Final dataset saved as finalizedData/finalizedData.csv
(2749, 17)
['host_is_superhost', 'host_listings_count', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'instant_bookable', 'amenities_count', 'price_category', 'cal_availability_rate', 'review_count']
